In [15]:
import random
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import pandas as pd
import os
from pathlib import Path
import xml.etree.ElementTree as ET

In [97]:
def filelist(root, file_type):
    return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]


def load_data (im_path, an_path):
    annotations = filelist(an_path, '.xml')
    data = []
    
    for an_path in annotations:
        tree = ET.parse(an_path)
        root = tree.getroot()
        width = int(root.find("./size/width").text)
        height = int(root.find("./size/height").text)
        image_path = root.find("./filename").text
        
        class_id = 0

        # READ ALL 'OBJECT' ELEMENTS FROM .XML FILE
        for obj in tree.findall('object'):        
            class_name = obj.find("./name").text
            xmin = int(obj.find("./bndbox/xmin").text)
            xmax = int(obj.find("./bndbox/xmax").text)
            ymin = int(obj.find("./bndbox/ymin").text)
            ymax = int(obj.find("./bndbox/ymax").text)
            
            if abs(xmax-xmin) > 0.1 * width and abs(ymax-ymin) > 0.1 * height and class_name == "crosswalk":
                class_id = 1
            
        image = cv2.imread(os.path.join(im_path, image_path))
        data.append({'image': image, 'label': class_id})
    
    return data

In [ ]:
def learn_bovw(data):
    #Learns BoVW dictionary and saves it as "voc.npy" file.

    dict_size = 128
    bow = cv2.BOWKMeansTrainer(dict_size)

    sift = cv2.SIFT_create()
    for sample in data:
        kpts = sift.detect(sample['image'], None)
        kpts, desc = sift.compute(sample['image'], kpts)

        if desc is not None:
            bow.add(desc)

    vocabulary = bow.cluster()

    np.save('voc.npy', vocabulary)

In [ ]:
def extract_features(data):
    #Extracts features for given data and saves it as "desc" entry.

    sift = cv2.SIFT_create()
    flann = cv2.FlannBasedMatcher_create()
    bow = cv2.BOWImgDescriptorExtractor(sift, flann)
    vocabulary = np.load('voc.npy')
    bow.setVocabulary(vocabulary)

    for sample in data:
        # compute descriptor and add it as "desc" entry in sample
        kpts = sift.detect(sample['image'], None)
        desc = bow.compute(sample['image'], kpts)
        sample['desc'] = desc

    return data

In [ ]:
def train(data):
    #Trains Random Forest classifier.

    # train random forest model and return it from function.
    descs = []
    labels = []

    for sample in data:
        if sample['desc'] is not None:
            print(sample['desc'].squeeze(0))
            descs.append(sample['desc'].squeeze(0))
            labels.append(sample['label'])

    clf = RandomForestClassifier()
    clf.fit(descs, labels)

    return clf

In [ ]:
def predict(rf, data):
    #Predicts labels given a model and saves them as "label_pred" (int) entry for each sample.

    # perform prediction using trained model and add results as "label_pred" (int) entry in sample

    for sample in data:
        if sample['desc'] is not None:
            pred = rf.predict(sample['desc'])
            sample['label_pred'] = int(pred)

    return data

In [ ]:
def evaluate(data):
    #Evaluates results of classification.

    # evaluate classification results and print statistics
    pred_labels = []
    true_labels=[]

    l = 0
    m = 0
    for sample in data:
        if sample['desc'] is not None:
            pred_labels.append(sample['label_pred'])
            true_labels.append(sample['label'])
            if sample['label'] == sample['label_pred']:
                l = l + 1
            else:
                m = m + 1

    print('accuracy= %.3f' % l/(l+m))

    matrix = confusion_matrix(true_labels, pred_labels)
    print(matrix)

    # this function does not return anything
    return

In [ ]:
def balance_dataset(data, ratio):
    #Subsamples dataset according to ratio.

    sampled_data = random.sample(data, int(ratio * len(data)))

    return sampled_data


In [98]:
train_im_path = Path('train/images')
train_an_path = Path('train/annotations')

data_train = load_data(train_im_path, train_an_path)

road156.png 1
road157.png 0
road158.png 1
road159.png 0
road160.png 1
road161.png 0
road162.png 0
road163.png 0
road164.png 0
road165.png 1
road166.png 1
road167.png 1
road168.png 0
road169.png 0
road170.png 0
road171.png 0
road172.png 0
road173.png 0
road174.png 0
road175.png 0
road176.png 1
road177.png 0
road178.png 1
road179.png 0
road180.png 1
road181.png 0
road182.png 0
road183.png 1
road184.png 0
road185.png 0
road186.png 0
road187.png 0
road188.png 0
road189.png 1
road190.png 1
road191.png 0
road192.png 0
road193.png 1
road194.png 1
road195.png 0
road196.png 0
road197.png 0
road198.png 0
road199.png 0
road200.png 1
road201.png 0
road202.png 0
road203.png 0
road204.png 1
road205.png 1
road206.png 0
road207.png 1
road208.png 0
road209.png 0
road210.png 0
road211.png 0
road212.png 0
road213.png 0
road214.png 0
road215.png 0
road216.png 0
road217.png 0
road218.png 0
road219.png 0
road220.png 0
road221.png 0
road222.png 0
road223.png 0
road224.png 0
road225.png 0
road226.png 1
road22

road750.png 0
road751.png 0
road752.png 0
road753.png 0
road754.png 0
road755.png 0
road756.png 0
road757.png 0
road758.png 0
road759.png 0
road760.png 0
road761.png 0
road762.png 0
road763.png 0
road764.png 0
road765.png 0
road766.png 0
road767.png 0
road768.png 0
road769.png 0
road770.png 0
road771.png 0
road772.png 0
road773.png 0
road774.png 0
road775.png 0
road776.png 0
road777.png 0
road778.png 0
road779.png 0
road780.png 0
road781.png 0
road782.png 0
road783.png 0
road784.png 1
road785.png 0
road786.png 0
road787.png 0
road788.png 0
road789.png 0
road790.png 0
road791.png 0
road792.png 0
road793.png 0
road794.png 0
road795.png 0
road796.png 0
road797.png 0
road798.png 0
road799.png 0
road800.png 0
road801.png 0
road802.png 0
road803.png 0
road804.png 0
road805.png 0
road806.png 0
road807.png 0
road808.png 0
road809.png 0
road810.png 0
road811.png 0
road812.png 0
road813.png 0
road814.png 0
road815.png 0
road816.png 0
road817.png 0
road818.png 0
road819.png 0
road820.png 0
road82

In [ ]:
data_train = balance_dataset(data_train, 1.0)

test_im_path = Path('test/images')
test_an_path = Path('test/annotations')
data_test = load_data(test_im_path, test_an_path)

data_test = balance_dataset(data_test, 1.0)

In [ ]:
# you can comment those lines after dictionary is learned and saved to disk.
    #print('learning BoVW')
    #learn_bovw(data_train)

In [ ]:
print('extracting train features')
#data_train = extract_features(data_train)

print('training')
rf = train(data_train)

In [ ]:
print('extracting test features')
data_test = extract_features(data_test)

In [ ]:
print('testing on testing dataset')
data_test = predict(rf, data_test)
evaluate(data_test)